<a href="https://colab.research.google.com/github/Aydin-ab/CV_DMTet/blob/main/DMTet_Tut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Dependencies: Kaolin and PyTorch3D

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from pathlib import Path
INSTALL_PATH = Path("/content/drive/'MyDrive'/CV_DMTet/")
%cd  $INSTALL_PATH

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CV_DMTet


In [ ]:
# reinstall cython, install usd-core (for 3D rendering), and clone into kaolin repo
!pip uninstall Cython --yes
!pip install Cython==0.29.20  --quiet
!pip install usd-core --quiet

Found existing installation: Cython 0.29.20
Uninstalling Cython-0.29.20:
  Successfully uninstalled Cython-0.29.20


In [ ]:
# installing kaolin and check version
%env IGNORE_TORCH_VER=1
%env KAOLIN_INSTALL_EXPERIMENTAL=1
KAOLIN_PATH = INSTALL_PATH / "kaolin"
%cd $INSTALL_PATH
!if [ ! -d $KAOLIN_PATH ]; then git clone --recursive https://github.com/NVIDIAGameWorks/kaolin; fi;
%cd $KAOLIN_PATH
SETUP_CHECK = KAOLIN_PATH / "kaolin" / "version.py"
# !echo Checking if $SETUP_CHECK exists
# !if [ ! -f $SETUP_CHECK ]; then python setup.py develop; fi;
!python setup.py develop
!python -c "import kaolin; print(kaolin.__version__)"

env: IGNORE_TORCH_VER=1
env: KAOLIN_INSTALL_EXPERIMENTAL=1
/content/drive/MyDrive/CV_DMTet
/content/drive/MyDrive/CV_DMTet/kaolin
setup.py:36: UserWarning: Kaolin is compatible with PyTorch >=1.6.0, <=1.13.0, but found version 1.13.0+cu116. Continuing with the installed version as IGNORE_TORCH_VER is set.
  warnings.warn(
Compiling kaolin/cython/ops/mesh/triangle_hash.pyx because it depends on /usr/local/lib/python3.8/dist-packages/Cython/Includes/libcpp/vector.pxd.
Compiling kaolin/cython/ops/conversions/mise.pyx because it depends on /usr/local/lib/python3.8/dist-packages/Cython/Includes/libcpp/vector.pxd.
[1/2] Cythonizing kaolin/cython/ops/conversions/mise.pyx
/usr/local/lib/python3.8/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/drive/MyDrive/CV_DMTet/kaolin/kaolin/cython/ops/conversions/mise.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[2/2]

# Import packages

In [ ]:
import numpy as np
import torch
import kaolin
import sys
import os

need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("1.12.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

from kaolin.ops.conversions import (
    trianglemeshes_to_voxelgrids,
    marching_tetrahedra,
    voxelgrids_to_cubic_meshes,
    voxelgrids_to_trianglemeshes,
)

from kaolin.ops.mesh import (
    index_vertices_by_faces
)

from kaolin.io.shapenet import (
    ShapeNetV2
)

from kaolin.metrics.trianglemesh import (
    point_to_mesh_distance,

)

from torch.utils.data import DataLoader

# add path for demo utils functions 
sys.path.append(os.path.abspath(''))
sys.path.append('/content/drive/MyDrive/CV_DMTet/')

# Import Dataset: Subset of ShapeNetV2

In [ ]:
import pytorch3d

In [ ]:
state_dict = torch.load('/content/drive/MyDrive/CV_DMTet/shapenet.pvcnn.c1.pth.tar')
state_dict.keys()

dict_keys(['model'])

In [ ]:
sys.path.append('/content/drive/MyDrive/kaolin/examples/')
sys.path.append('/content/drive/MyDrive/kaolin/examples/tutorial')

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")
device

device(type='cuda', index=0)

In [ ]:
# path to the point cloud to be reconstructed
pcd_path = "/content/drive/MyDrive/kaolin/examples/samples/bear_pointcloud.usd"
# path to the output logs (readable with the training visualizer in the omniverse app)
logs_path = '/content/drive/MyDrive/CV_DMTet/Logs'

# We initialize the timelapse that will store USD for the visualization apps
timelapse = kaolin.visualize.Timelapse(logs_path)

#Load Tetrahedral grid

DMTet starts from a uniform tetrahedral grid of predefined resolution, and uses a network to predict the SDF value as well as deviation vector at each grid vertex.

Here we load the pre-generated tetrahedral grid using Quartet at resolution 128, which has roughly the same number of vertices as a voxel grid of resolution 65. We use a simple MLP + positional encoding to predict the SDF and deviation vectors in DMTet, and initialize the encoded SDF to represent a sphere.

In [ ]:
# Uniform Tetrahedral Grid
tets_verts = torch.tensor(np.load('/content/drive/MyDrive/kaolin/examples/samples/128_verts.npz')['data'], dtype=torch.float, device=device)
tets = torch.tensor(([np.load('/content/drive/MyDrive/kaolin/examples/samples/128_tets_{}.npz'.format(i))['data'] for i in range(4)]), dtype=torch.long, device=device).permute(1,0)
print(tets_verts, tets)

tensor([[ 0.5000,  0.5000,  0.4844],
        [ 0.4844,  0.5000,  0.4922],
        [ 0.4922,  0.4844,  0.4844],
        ...,
        [-0.1719, -0.5000,  0.4766],
        [-0.1562, -0.5000,  0.4688],
        [-0.1562, -0.4922,  0.4688]], device='cuda:0') tensor([[     0,      1,      2,      3],
        [     2,      3,      1,      4],
        [     5,      3,      0,      2],
        ...,
        [277409, 272920, 272914, 272919],
        [272919, 277409, 272920, 274866],
        [277409, 277400, 272920, 274866]], device='cuda:0')


<ipython-input-10-6909fe0ff698>:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  tets = torch.tensor(([np.load('/content/drive/MyDrive/kaolin/examples/samples/128_tets_{}.npz'.format(i))['data'] for i in range(4)]), dtype=torch.long, device=device).permute(1,0)


# Loading from ShapeNet

In [ ]:
SHAPENET_PATH = "/content/drive/MyDrive/CV_DMTet/Core"
#SHAPENET_PATH = "/content/drive/MyDrive/FALL 2022/Computer Vision/Project/Core"
SYNSETS_IDS = ['02747177', '02773838', '02801938', '02808440', '02818832', '02828884', '02843684', '02871439', '02876657', '02880940', '02924116', '02933112']
shapenet_train = ShapeNetV2(SHAPENET_PATH, categories=SYNSETS_IDS, output_dict=True)
shapenet_test = ShapeNetV2(SHAPENET_PATH, categories=SYNSETS_IDS, output_dict=True, train=False)

In [ ]:
sample_model = shapenet_train[2000] # change the index here for different models
sample_verts = sample_model['mesh'][0]
sample_faces = sample_model['mesh'][1]

timelapse.add_mesh_batch(
    category='gt',
    vertices_list=[sample_verts.cpu()],
    faces_list=[sample_faces.cpu()]
)

# Convert model to watertight meshes

We used a voxelization with resolution of 64 to predict the sdf and extract surface. 

In [ ]:
wt_grid = kaolin.ops.conversions.trianglemeshes_to_voxelgrids(
    vertices=sample_verts.unsqueeze(0).to(device),
    faces=sample_faces.to(device),
    resolution=64
)

In [ ]:
wt_verts, wt_faces = kaolin.ops.conversions.voxelgrids_to_cubic_meshes(wt_grid)
wt_verts, wt_faces = wt_verts[0], wt_faces[0]

In [ ]:
# wt_verts = kaolin.metrics.trianglemesh.uniform_laplacian_smoothing(wt_verts[0].unsqueeze(0), wt_faces[0])
# sample_verts, sample_faces = wt_verts[0], wt_faces[0]

In [ ]:
center = (wt_verts.max(0)[0] + wt_verts.min(0)[0]) / 2
max_l = (wt_verts.max(0)[0] - wt_verts.min(0)[0]).max()
wt_verts = ((wt_verts - center) / max_l)* 0.9
timelapse.add_mesh_batch(
    category='watertight_test',
    vertices_list=[wt_verts.cpu()],
    faces_list=[wt_faces.cpu()]
)

# SDF model

We follow the paper recommandation and use a four-layer
MLPs with hidden dimensions 256, 256, 128 and 64, respectively

In [ ]:
# Since we skip PVCNN, input dimension is just the coordinates of each grid

SDF_MLP_CONFIG = {
    'input_dim' : 3, # Coordinates of the grid's vertices
    'hidden_dims' : [256, 256, 128, 64],
    'output_dim' : 1, # SDF of the vertex input. The other "output" f_v comes from the prior activation layer of dimension 64
    'multires': 2
}

lr = 0.01
laplacian_weight = 0.1
iterations = 2000
save_every = 100
multires = 2
grid_res = 128

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

class MLP(torch.nn.Module):
    def __init__(self, config):
        super(MLP, self).__init__()

        self.input_dim = config['input_dim']
        self.hidden_dims  = config['hidden_dims']
        self.output_dim = config['output_dim']
        self.multires = config['multires']

        self.embed_fn = None
        # if self.multires > 0:
        #     embed_fn, input_ch = get_embedder(self.multires)
        #     self.embed_fn = embed_fn
        #     self.input_dim = input_ch

        # Hidden layers
        self.hiddens = nn.ModuleList()
        in_dim = self.input_dim
        for k in range(len(self.hidden_dims)):
            self.hiddens.append(nn.Linear(in_dim, self.hidden_dims[k]))
            in_dim = self.hidden_dims[k]

        # Output layer
        self.output_layer = torch.nn.Linear(self.hidden_dims[-1], self.output_dim)


    def forward(self, x):
        if self.embed_fn is not None:
            x = self.embed_fn(x)
        for hidden in self.hiddens :
            x = F.relu(hidden(x))
        output = self.output_layer(x) # No activation (linear) cuz we do regression

        return output, x # Return sdf predicted + f_v feature vector


# Positional Encoding from https://github.com/yenchenlin/nerf-pytorch/blob/1f064835d2cca26e4df2d7d130daa39a8cee1795/run_nerf_helpers.py
class Embedder:
    def __init__(self, **kwargs):
        self.kwargs = kwargs
        self.create_embedding_fn()
        
    def create_embedding_fn(self):
        embed_fns = []
        d = self.kwargs['input_dims']
        out_dim = 0
        if self.kwargs['include_input']:
            embed_fns.append(lambda x : x)
            out_dim += d
            
        max_freq = self.kwargs['max_freq_log2']
        N_freqs = self.kwargs['num_freqs']
        
        if self.kwargs['log_sampling']:
            freq_bands = 2.**torch.linspace(0., max_freq, steps=N_freqs)
        else:
            freq_bands = torch.linspace(2.**0., 2.**max_freq, steps=N_freqs)
            
        for freq in freq_bands:
            for p_fn in self.kwargs['periodic_fns']:
                embed_fns.append(lambda x, p_fn=p_fn, freq=freq : p_fn(x * freq))
                out_dim += d
                    
        self.embed_fns = embed_fns
        self.out_dim = out_dim
        
    def embed(self, inputs):
        return torch.cat([fn(inputs) for fn in self.embed_fns], -1)

def get_embedder(multires):
    embed_kwargs = {
                'include_input' : True,
                'input_dims' : 3,
                'max_freq_log2' : multires-1,
                'num_freqs' : multires,
                'log_sampling' : True,
                'periodic_fns' : [torch.sin, torch.cos],
    }
    
    embedder_obj = Embedder(**embed_kwargs)
    embed = lambda x, eo=embedder_obj : eo.embed(x)
    return embed, embedder_obj.out_dim

In [ ]:
sdf_model = MLP(SDF_MLP_CONFIG).to(device)
print(sdf_model)
print('\n\n')
summary(sdf_model, input_size= tets_verts.shape)

MLP(
  (hiddens): ModuleList(
    (0): Linear(in_features=3, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): Linear(in_features=128, out_features=64, bias=True)
  )
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
)



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1          [-1, 277410, 256]           1,024
            Linear-2          [-1, 277410, 256]          65,792
            Linear-3          [-1, 277410, 128]          32,896
            Linear-4           [-1, 277410, 64]           8,256
            Linear-5            [-1, 277410, 1]              65
Total params: 108,033
Trainable params: 108,033
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 3.17
Forward/backward pass size (MB): 1492.11

Little Test:

In [ ]:
pred_sdfs, f_vs = sdf_model(tets_verts)

print(f'Input grid shape is : {tets_verts.shape}')
print(f'Output shape of the predicted SDFs should be {tets_verts.shape[0], 1} and it actually is {tuple(pred_sdfs.shape)}')
print(f'Output shape of the feature vectors f_vs should be {tets_verts.shape[0], 64} and it actually is {tuple(f_vs.shape)}')
print(pred_sdfs)

Input grid shape is : torch.Size([277410, 3])
Output shape of the predicted SDFs should be (277410, 1) and it actually is (277410, 1)
Output shape of the feature vectors f_vs should be (277410, 64) and it actually is (277410, 64)
tensor([[0.1024],
        [0.1021],
        [0.1022],
        ...,
        [0.1006],
        [0.1006],
        [0.1006]], device='cuda:0', grad_fn=<AddmmBackward0>)


# Set up Optimizer

In [ ]:
sdf_vars = [p for _, p in sdf_model.named_parameters()]
sdf_optimizer = torch.optim.Adam(sdf_vars, lr=lr)
sdf_scheduler = torch.optim.lr_scheduler.LambdaLR(sdf_optimizer, lr_lambda=lambda x: max(0.0, 10**(-x*0.0002))) # LR decay over time

# Training

In [ ]:
# takes in a module and applies the specified weight initialization
def weights_init_normal(m):
    '''Takes in a module and initializes all linear layers with weight
        values taken from a normal distribution.'''

    classname = m.__class__.__name__
    # for every Linear layer in a model
    if classname.find('Linear') != -1:
        y = m.in_features
    # m.weight.data shoud be taken from a normal distribution
        m.weight.data.normal_(0.0,1/np.sqrt(y))
    # m.bias.data should be 0
        m.bias.data.fill_(0)

In [ ]:
sdf_model.apply(weights_init_normal)

MLP(
  (hiddens): ModuleList(
    (0): Linear(in_features=3, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): Linear(in_features=128, out_features=64, bias=True)
  )
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
)

In [ ]:
def sdf_train(iterations, model, optimizer, scheduler):
  model.train()
  gt_verts = wt_verts.to(device)
  gt_faces = wt_faces.to(device)

  f = index_vertices_by_faces(gt_verts.unsqueeze(0), gt_faces)
  gt_sdfs, idx, t = point_to_mesh_distance(tets_verts.unsqueeze(0), f)

  sign = kaolin.ops.mesh.check_sign(gt_verts.unsqueeze(0).to(device), gt_faces.to(device), tets_verts.unsqueeze(0))
  gt_sdfs[sign==False] *= -1

  for it in range(iterations):
      pred_sdfs, f_vs = model(tets_verts)
      
      loss = F.mse_loss(pred_sdfs, gt_sdfs.squeeze(0).unsqueeze(1), reduction='sum')

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      scheduler.step()

      if (it) % save_every == 0 or it == (iterations - 1): 
          print ('Iteration {} - loss: {}'.format(it, loss))


In [ ]:
# ~12 min. Speed up?
sdf_train(iterations, sdf_model, sdf_optimizer, sdf_scheduler)

Iteration 0 - loss: 0.011151010170578957
Iteration 100 - loss: 5.3956679039401934e-05
Iteration 200 - loss: 3.137449675705284e-05
Iteration 300 - loss: 2.4316572307725437e-05
Iteration 400 - loss: 1.9445456928224303e-05
Iteration 500 - loss: 1.7838116036728024e-05
Iteration 600 - loss: 1.8048469428322278e-05
Iteration 700 - loss: 1.5522813555435278e-05
Iteration 800 - loss: 1.5283503671525978e-05
Iteration 900 - loss: 1.4500064025924075e-05
Iteration 1000 - loss: 1.572274231875781e-05
Iteration 1100 - loss: 1.3757760825683363e-05
Iteration 1200 - loss: 1.3320885045686737e-05
Iteration 1300 - loss: 1.3232168385002296e-05
Iteration 1400 - loss: 1.2258676179044414e-05
Iteration 1500 - loss: 1.1962479220528621e-05
Iteration 1600 - loss: 1.1679520866891835e-05
Iteration 1700 - loss: 1.1446755706856493e-05
Iteration 1800 - loss: 1.1237471881031524e-05
Iteration 1900 - loss: 1.1052374247810803e-05
Iteration 1999 - loss: 1.2083042747690342e-05


 # Surface refinement utils

In [ ]:

# Get edges lists of shape [E, 2] from face list of shape [V, 4]

def get_edges(input):
  c = torch.combinations(torch.arange(input.size(1)), r=2)
  x = input[:,None].expand(-1,len(c),-1).cpu()
  idx = c[None].expand(len(x), -1, -1)
  x = x.gather(dim=2, index=idx)

  return x.view(-1, *x.shape[2:])

# Extract tets under certain sdf restrictions:
# if thresh = 0, return all surface tetrahedrons
# if thresh > 0, return all tetrahedrons whose vertices' sdfs are all in the range [-thresh, thresh]

def extract_tet(tets, sdf, thresh, non_surf=False):

  assert thresh >= 0

  if thresh == 0:
    mask = sdf[tets] > 0
    mask_int = mask.squeeze(2).long()
    t = mask_int.sum(1)
    surf_tets = tets[(t > 0) & (t < 4)]
  else:
    mask = (sdf[tets] >= -thresh) & (sdf[tets] <= thresh)
    mask_int = mask.squeeze(2).long()
    t = mask_int.sum(1)
    surf_tets = tets[t == 4]

  surf_tets_tuple = surf_tets.unique(return_inverse=True)
  surf_tets_idx, surf_tets = surf_tets_tuple[0], surf_tets_tuple[1]

  if non_surf:
    non_surf_tets = tets[t == 0]
    non_surf_tets_tuple = non_surf_tets.unique(return_inverse=True)
    non_surf_tets_idx, non_surf_tets = non_surf_tets_tuple[0], non_surf_tets_tuple[1]
    return surf_tets_idx, surf_tets, non_surf_tets_idx, non_surf_tets

  return surf_tets_idx, surf_tets

#Get output from initial MLP

def get_pred_sdfs(model, tets_verts):
  model.eval()

  with torch.no_grad():
    pred_sdfs, f_vs = model(tets_verts)

  return pred_sdfs, f_vs

# Get ground truth sdf from input verts and faces
# input shape: [batch_size, num_vertices, 3], [num_faces, 4], [batch_size, num_points, 3]
# output shape: [num_points, 1]

def get_gt_sdfs(gt_verts, gt_faces, points, f=None):

  if f == None:
    f = index_vertices_by_faces(gt_verts, gt_faces)
  d,_,_ = point_to_mesh_distance(points, f)

  s = kaolin.ops.mesh.check_sign(gt_verts, gt_faces, points)
  d[s==True] *= -1

  return d.squeeze(0).unsqueeze(1)

# Check quality of sdf prediction

In [ ]:
d = get_gt_sdfs(wt_verts.unsqueeze(0).to(device), wt_faces.to(device), tets_verts.unsqueeze(0))

In [ ]:
print(d[d > 0])

tensor([0.1024, 0.1021, 0.1022,  ..., 0.1006, 0.1006, 0.1006], device='cuda:0')
tensor([0.2277, 0.2311, 0.2158,  ..., 0.2419, 0.2374, 0.2313], device='cuda:0')


In [ ]:
surf_tets_verts_idx, surf_tets_faces = extract_tet(tets, d, 0)
surf_tets_verts = tets_verts[surf_tets_verts_idx]

timelapse.add_mesh_batch(
            category='tet_surface',
            vertices_list=[surf_tets_verts.cpu()],
            faces_list=[surf_tets_faces.cpu()]
        )

# Surface refinement model

In [ ]:
"""
Graph-res net:
Identify surface tetrahedral, build adj matrix, 
"""

from tqdm import tqdm

from torch import nn
from torch.nn import functional as F
from pytorch3d.ops import GraphConv

# a single res block layer with dimension 256 & 128
class GResBlock(nn.Module): 
    def __init__(self, in_dim, hidden_dim, activation=None):
        super(GResBlock, self).__init__()

        self.conv1 = GraphConv(in_dim, hidden_dim)
        self.conv2 = GraphConv(hidden_dim, in_dim)
        self.activation = F.relu if activation else None
    
    def forward(self, inputs):
        input, adj = inputs[0], inputs[1]
        x = self.conv1(input, adj)
        if self.activation:
          x = self.activation(x)
        x = self.conv2(x, adj)
        if self.activation:
          x = self.activation(input + x)
        
        return [x, adj]

class GBottleneck(nn.Module):
    def __init__(self, block_num, in_dim, hidden_dim, out_dim, activation=None):
        super(GBottleneck, self).__init__()

        resblock_layers = [GResBlock(in_dim=hidden_dim[0], hidden_dim=hidden_dim[1], activation=activation)
                          for _ in range(block_num)]
        self.blocks = nn.Sequential(*resblock_layers)
        self.conv1 = GraphConv(in_dim, hidden_dim[0])

        self.activation = F.relu if activation else None
    
    def forward(self, inputs, adj):
        x = self.conv1(inputs, adj)
        if self.activation:
          x = self.activation(x)
        x = self.blocks([x, adj])[0]
        if self.activation:
          x = self.activation(x)

        return x

class GCN_Res(nn.Module):
    def __init__(self, config):
        super(GCN_Res, self).__init__()

        self.in_dim = config['in_dim']
        self.hidden_dim = config['hidden_dim']
        self.out_dim = config['out_dim']
        self.activation = config['activation']
        self.mlp_hdim = config['mlp_hdim']
        self.mlp_odim = config['mlp_odim']

        self.gcn_res = nn.ModuleList([GBottleneck(2, self.in_dim, self.hidden_dim, self.out_dim, self.activation)])

        self.sdf_mlp = nn.Sequential(
            nn.Linear(self.out_dim, self.mlp_hdim[0], bias=False),
            nn.Linear(self.mlp_hdim[0], self.mlp_hdim[1], bias=False),
            nn.Linear(self.mlp_hdim[1], 1, bias=False),
        )

        self.deform_mlp = nn.Sequential(
            nn.Linear(self.out_dim, self.mlp_hdim[0], bias=False),
            nn.Linear(self.mlp_hdim[0], self.mlp_hdim[1], bias=False),
            nn.Linear(self.mlp_hdim[1], 3, bias=False),
        )

        self.feature_mlp = nn.Sequential(
            nn.Linear(self.out_dim, self.mlp_hdim[0], bias=False),
            nn.Linear(self.mlp_hdim[0], self.mlp_hdim[1], bias=False),
            nn.Linear(self.mlp_hdim[1], self.mlp_hdim[1], bias=False),
        )
        


    def forward(self, inputs, adj):

        x = self.gcn_res[0](inputs, adj)

        sdf = self.sdf_mlp(x)
        deform = self.deform_mlp(x)
        feature = self.feature_mlp(x)
        
        return sdf, deform, feature


In [ ]:
def gcn_loss(iterations, mesh_verts, mesh_faces, gt_verts, gt_faces, it):

    #surface alignment loss
    m = pytorch3d.structures.Meshes([mesh_verts, gt_verts], [mesh_faces, gt_faces])
  
    # pc = pytorch3d.ops.sample_points_from_meshes(m, num_samples=50000)
    pred_points = kaolin.ops.mesh.sample_points(mesh_verts.unsqueeze(0), mesh_faces, 50000)[0][0]
    gt_points = kaolin.ops.mesh.sample_points(gt_verts.unsqueeze(0), gt_faces, 50000)[0][0]

    # pred_points,gt_points = pc[0], pc[1]
    chamfer = kaolin.metrics.pointcloud.chamfer_distance(pred_points.unsqueeze(0), gt_points.unsqueeze(0)).mean()
    
    norm = pytorch3d.loss.mesh_normal_consistency(m)
    if it > iterations//2:
      lap = pytorch3d.loss.mesh_laplacian_smoothing(m)
      return 500*chamfer + lap * laplacian_weight + 1e-6*norm # + sdf_loss + deform_loss
    return 500*chamfer + 1e-6*norm #+ sdf_loss + deform_loss

# Train GCN Model for Surface Refinement

In [ ]:
CONFIG_GCNRES = {
    'in_dim': 68,
    'hidden_dim': [128, 256],
    'out_dim': 128,
    'activation': True,
    'mlp_hdim': [128,64],
    'mlp_odim': 68,
}

# Same set of Hyperparam is applied
lr = 1e-4
laplacian_weight = 0.1
gcn_iterations = 5000
save_every = 100
multires = 2
grid_res = 128

In [ ]:
from tqdm import tqdm

def gcn_pretrain(iterations, gcn_model, optimizer, pred_sdf, tets_verts, tets, tets_verts_features, gt_verts, gt_faces, thresh):
  gcn_model.train()

  assert thresh >= 0

  surf_tets_verts_idx, surf_tets_faces = extract_tet(tets, pred_sdf, thresh)
  surf_tets_verts = torch.clone(tets_verts[surf_tets_verts_idx])

  surf_tets_verts_features = torch.clone(tets_verts_features[surf_tets_verts_idx])
  surf_sdfs = pred_sdf[surf_tets_verts_idx].detach()
  surf_tets_edges = torch.clone(get_edges(surf_tets_faces).to(device))
  
  gt_verts, gt_faces = gt_verts.to(device), gt_faces.to(device)

  # f = index_vertices_by_faces(gt_verts.unsqueeze(0), gt_faces)

  for it in tqdm(range(iterations)):
    verts_f = torch.cat((surf_tets_verts, surf_sdfs, surf_tets_verts_features), dim=1)

    sdf, deform, fv = gcn_model(verts_f, surf_tets_edges)

    update_verts = surf_tets_verts + torch.tanh(deform) / grid_res
    update_sdfs = surf_sdfs + sdf

    gt_sdfs = get_gt_sdfs(gt_verts.unsqueeze(0), gt_faces, update_verts.unsqueeze(0))

    sdf_loss = F.mse_loss(update_sdfs, gt_sdfs, reduction='mean')

    optimizer.zero_grad()
    sdf_loss.backward(retain_graph=True)
    optimizer.step()

    if it == iterations-1:
      print('Iteration {} - loss: {}'.format(iterations, sdf_loss))


In [ ]:
"""
Identify surface Tetrahedral
input: f = [v, s, F_vol, f_64]
output: f = [delta v, delta s, f_64]

update v = v + delta v, s = s + delta s
generate new meshes by marching tet from v and s; calculate loss

loss = chamfer + sdf_loss + deform_loss
"""
from pytorch3d import loss

def gcn_train(iterations, gcn_model, optimizer, scheduler, pred_sdf, tets_verts, tets, tets_verts_features, gt_verts, gt_faces, thresh):
  gcn_model.train()

  assert thresh >= 0

  """

  Firstly, based on predicted sdf value, extract out all surface tetrahedrons
  i.e. 1-3 vertices of tetrahedrons are with different sdf signs
  return value: 
  --> surf_tets_idx: origin index of vertices as in tet grid
  --> surf_tets_faces: reindexed faces list start from 0

  """

  surf_tets_verts_idx, surf_tets_faces, non_surf_idx, non_surf_tets = extract_tet(tets, pred_sdf, thresh, True)
  surf_tets_edges = get_edges(surf_tets_faces).to(device)

  surf_tets_verts = torch.clone(tets_verts[surf_tets_verts_idx])

  # obtain per vertex features: 
  # sdf, position, feature vector

  surf_tets_verts_features = torch.clone(tets_verts_features[surf_tets_verts_idx])
  surf_sdfs = pred_sdf[surf_tets_verts_idx].detach()
  surf_verts_f = torch.cat((surf_tets_verts, surf_sdfs, surf_tets_verts_features), dim=1)

  non_surf_verts = torch.clone(tets_verts[non_surf_idx])

  # calculate ground truth sdf value over all vertices in tet mesh

  # gt_f = index_vertices_by_faces(gt_verts.unsqueeze(0), gt_faces).to(device)
  # gt_sdfs, _, _ = point_to_mesh_distance(tets_verts.unsqueeze(0), gt_f)
  # gt_sdfs = gt_sdfs.squeeze(0).unsqueeze(1)

  gt_verts, gt_faces = gt_verts.to(device), gt_faces.to(device)

  gt_f = index_vertices_by_faces(gt_verts.unsqueeze(0), gt_faces)
  
  gt_sdfs = get_gt_sdfs(gt_verts.unsqueeze(0), gt_faces, tets_verts.unsqueeze(0), gt_f)

  for it in range(iterations):
    
    """
    
    Secondly, predict with a 2-layer GCN followed by 2-layer MLP
    output:
    --> [delta v, delta s, new f_s] \in R^68

    """

    sdf, deform, fv = gcn_model(surf_verts_f, surf_tets_edges)
    
    """

    Update surface position, sdf, and f_s

    """

    #updated sdf

    us = sdf.detach()
    update_sdfs = torch.clone(pred_sdf)
    update_sdfs[surf_tets_verts_idx] += us

    #update vertices positions

    vd = torch.tanh(deform).detach()
    update_tets_verts = torch.clone(tets_verts)
    update_tets_verts[surf_tets_verts_idx] += vd / grid_res

    #update vertices features

    vfvs = fv.detach()
    update_tets_f = torch.clone(tets_verts_features)
    update_tets_f[surf_tets_verts_idx] += vfvs

    """

    Marching Tetrahedra based on new sdf value and deformed vertices in the tet grid

    """

    mesh_verts, mesh_faces = kaolin.ops.conversions.marching_tetrahedra(update_tets_verts.unsqueeze(0), tets, update_sdfs.squeeze(1).unsqueeze(0))
    mesh_verts, mesh_faces = mesh_verts[0], mesh_faces[0]

    """

    Compute Loss for First surface refinement: 
    Normal consistency + surface alignment + laplacian smooth + sdf L2-reg + deform L2-reg

    """

    # L2 sdf reg: 

    # Calculate SDF non_surface tetrahedron in newly generated mesh

    p_sdfs = get_gt_sdfs(mesh_verts.unsqueeze(0), mesh_faces, non_surf_verts.unsqueeze(0))

    # Get ground truth SDF value of non_surface tetrahedron vertices and truncate at +- 0.3 to focus on surface

    ns_g = gt_sdfs[non_surf_idx]
    mask = (ns_g >= -0.3) & (ns_g <= 0.3)
    p = p_sdfs[mask]
    g = ns_g[mask]

    # Also calculate surface sdf loss

    s_sdfs = get_gt_sdfs(gt_verts.unsqueeze(0), gt_faces, update_tets_verts[surf_tets_verts_idx].unsqueeze(0), gt_f)

    sdf_loss = F.mse_loss(sdf, s_sdfs - surf_sdfs, reduction='sum') #+F.mse_loss(p, g, reduction='sum') +

    #L2 deform reg

    deform_loss = F.mse_loss(deform, torch.zeros(deform.shape).to(device), reduction='sum')

    #surface alignment loss

    r_loss = gcn_loss(iterations, mesh_verts, mesh_faces, gt_verts, gt_faces, it)

    loss = r_loss + 0.4*sdf_loss + deform_loss

    optimizer.zero_grad()
    torch.autograd.set_detect_anomaly(True)
    loss.backward(retain_graph=True)
    optimizer.step()
    scheduler.step()

    if (it) % save_every == 0 or it == (iterations - 1): 
      print ('Iteration {} - loss: {}, # of mesh vertices: {}, # of mesh faces: {}'.format(it, loss, mesh_verts.shape[0], mesh_faces.shape[0]))
      
      # save reconstructed mesh
      timelapse.add_mesh_batch(
          iteration=it+1,
          category='extracted_mesh',
          vertices_list=[mesh_verts.cpu()],
          faces_list=[mesh_faces.cpu()]
      )

In [ ]:
from pytorch3d import loss

def gcn_train_wsubdiv(iterations, gcn_model, optimizer, scheduler, pred_sdf, tets_verts, tets, tets_verts_features, gt_verts, gt_faces, thresh):
  gcn_model.train()

  assert thresh >= 0

  """

  Firstly, based on predicted sdf value, extract out all surface tetrahedrons
  i.e. 1-3 vertices of tetrahedrons are with different sdf signs
  return value: 
  --> surf_tets_idx: origin index of vertices as in tet grid
  --> surf_tets_faces: reindexed faces list start from 0

  """

  # Get surface and non-surface tetrahedron indices

  surf_tets_verts_idx, surf_tets_faces, non_surf_idx, non_surf_tets = extract_tet(tets, pred_sdf, thresh, True)

  # Use surface faces to obtain edge list

  surf_tets_edges = get_edges(surf_tets_faces).to(device)


  # Use surface indices obtain per vertex features: 

  # Surface vertices positions:

  surf_tets_verts = torch.clone(tets_verts[surf_tets_verts_idx])

  # Surface per vertex feature vector:

  surf_tets_verts_features = torch.clone(tets_verts_features[surf_tets_verts_idx])

  # Surface per vertex predicted sdf value:

  surf_sdfs = torch.clone(pred_sdf[surf_tets_verts_idx])

  # Cat all features together as the input into model

  surf_verts_f = torch.cat((surf_tets_verts, surf_sdfs, surf_tets_verts_features), dim=1)

  # Obtain non_surface vertices position

  non_surf_verts = torch.clone(tets_verts[non_surf_idx])

  # Calculate ground truth sdf value over all vertices in tet mesh

  gt_verts, gt_faces = gt_verts.to(device), gt_faces.to(device)
  gt_f = index_vertices_by_faces(gt_verts.unsqueeze(0), gt_faces)

  gt_sdfs = get_gt_sdfs(gt_verts.unsqueeze(0), gt_faces, tets_verts.unsqueeze(0), gt_f)

  for it in range(100):
    
    """
    
    Secondly, predict with a 2-layer GCN followed by 2-layer MLP
    output:
    --> [delta v, delta s, new f_s] \in R^68

    """

    sdf, deform, fv = gcn_model(surf_verts_f, surf_tets_edges)
    
    """

    Update surface position, sdf, and f_s

    """

    #updated sdf

    us = sdf.detach()
    update_sdfs = torch.clone(pred_sdf)
    update_sdfs[surf_tets_verts_idx] += us

    #update vertices positions

    vd = torch.tanh(deform).detach()
    update_tets_verts = torch.clone(tets_verts)
    update_tets_verts[surf_tets_verts_idx] += vd / grid_res

    #update vertices features

    vfvs = fv.detach()
    update_tets_f = torch.clone(tets_verts_features)
    update_tets_f[surf_tets_verts_idx] += vfvs

    """

    Volume subdivision:
    Re-identify surface tets based on updated sdf value and subdivide identified surface tets


    """

    # Extract surface tetrahedrons based on updated sdf value

    d_surf_tets_verts_idx, d_surf_tets_faces = extract_tet(tets, update_sdfs, thresh)
    if d_surf_tets_faces.shape[0] == 0:
      d_surf_tets_verts_idx, d_surf_tets_faces = surf_tets_verts_idx, surf_tets_faces

    # Use new surface indices to get new surface vertices positions

    d_surf_tets_verts = torch.clone(update_tets_verts[d_surf_tets_verts_idx])

    # Use new surface indices to get new surface per vertex sdf and features

    d_surf_tets_verts_features = torch.clone(update_tets_f[d_surf_tets_verts_idx])
    d_surf_sdfs = torch.clone(update_sdfs[d_surf_tets_verts_idx])

    # Cat new surface per vertex features together for subdivide

    d_surf_verts_f = torch.cat((d_surf_sdfs, d_surf_tets_verts_features), dim=1)

    # Perform subdivide, returns subdivided surface tetrahedron 

    d_tets_verts, d_tets_faces, d_tets_fvs = kaolin.ops.mesh.subdivide_tetmesh(d_surf_tets_verts.unsqueeze(0), d_surf_tets_faces, d_surf_verts_f.unsqueeze(0))
    d_tets_verts, d_tets_fvs = d_tets_verts[0], d_tets_fvs[0]
    d_tets_sdfs = d_tets_fvs[:, 0]


    # Get surface edges from tetrahedron faces

    d_tets_edges = get_edges(d_tets_faces).to(device)

    # Cat to obtain feature input into GCN

    d_tets_fvs = torch.cat((d_tets_verts, d_tets_fvs), dim=1)

    d_sdf, d_deform, d_fv = gcn_model(d_tets_fvs, d_tets_edges)

    # Updated sdf

    d_update_sdfs = d_tets_sdfs.unsqueeze(1).detach() + d_sdf #since we need this to back propagate


    # Update vertices positions

    d_vd = torch.tanh(d_deform).detach()
    d_update_tets_verts = d_tets_verts + d_vd / grid_res

    # Update vertices features

    d_vfvs = d_fv.detach()
    d_update_tets_f = d_tets_fvs[:, 4:] + d_vfvs

    """

    Marching Tetrahedra based on new sdf value and deformed vertices in the tet grid

    """

    mesh_verts, mesh_faces = kaolin.ops.conversions.marching_tetrahedra(d_update_tets_verts.unsqueeze(0), d_tets_faces, d_update_sdfs.squeeze(1).unsqueeze(0))
    mesh_verts, mesh_faces = mesh_verts[0], mesh_faces[0]

    print(mesh_verts.shape, mesh_faces.shape)

    """

    Compute Loss for First surface refinement: 
    Normal consistency + surface alignment + laplacian smooth + sdf L2-reg + deform L2-reg

    """

    # L2 sdf reg: 

    # Calculate SDF non_surface tetrahedron in newly generated mesh

    p_sdfs = get_gt_sdfs(mesh_verts.unsqueeze(0), mesh_faces, non_surf_verts.unsqueeze(0))

    # Get ground truth SDF value of non_surface tetrahedron vertices and truncate at +- 0.3 to focus on surface

    ns_g = gt_sdfs[non_surf_idx]
    mask = (ns_g >= -0.3) & (ns_g <= 0.3)
    p = p_sdfs[mask]
    g = ns_g[mask]

    # Also calculate surface sdf loss

    # s_sdfs, _, _ = point_to_mesh_distance(update_tets_verts[surf_tets_verts_idx].unsqueeze(0), gt_f)
    # s_sign = kaolin.ops.mesh.check_sign(gt_verts.unsqueeze(0), gt_faces, update_tets_verts[surf_tets_verts_idx].unsqueeze(0))
    # s_sdfs[s_sign==False] *= -1

    # s_sdfs = s_sdfs.squeeze(0).unsqueeze(1)

    s_sdfs = get_gt_sdfs(gt_verts.unsqueeze(0), gt_faces, d_update_tets_verts.unsqueeze(0), gt_f)

    sdf_loss = F.mse_loss(p, g, reduction='sum') + F.mse_loss(d_update_sdfs, s_sdfs, reduction='sum')

    #L2 deform reg

    deform_loss = F.mse_loss(d_deform, torch.zeros(d_deform.shape).to(device), reduction='sum')

    #surface alignment loss

    r_loss = gcn_loss(iterations, mesh_verts, mesh_faces, gt_verts, gt_faces, it)
    
    loss = r_loss + deform_loss + 0.4*sdf_loss 

    optimizer.zero_grad()
    torch.autograd.set_detect_anomaly(True)
    loss.backward(retain_graph=True)
    optimizer.step()
    scheduler.step()

    # print('Iteration {} - loss: {}'.format(it, loss))

    if (it) % save_every == 0 or it == (iterations - 1): 
      print ('Iteration {} - loss: {}, # of mesh vertices: {}, # of mesh faces: {}'.format(it, loss, mesh_verts.shape[0], mesh_faces.shape[0]))
      
      # save reconstructed mesh
      timelapse.add_mesh_batch(
          iteration=it+1,
          category='extracted_mesh',
          vertices_list=[mesh_verts.cpu()],
          faces_list=[mesh_faces.cpu()]
      )

In [ ]:
refine_model = GCN_Res(CONFIG_GCNRES).to(device)

print(refine_model)

pre_vars = [p for _, p in refine_model.named_parameters()]
pre_optimizer = torch.optim.Adam(pre_vars, lr=lr)

GCN_Res(
  (gcn_res): ModuleList(
    (0): GBottleneck(
      (blocks): Sequential(
        (0): GResBlock(
          (conv1): GraphConv(128 -> 256, directed=False)
          (conv2): GraphConv(256 -> 128, directed=False)
        )
        (1): GResBlock(
          (conv1): GraphConv(128 -> 256, directed=False)
          (conv2): GraphConv(256 -> 128, directed=False)
        )
      )
      (conv1): GraphConv(68 -> 128, directed=False)
    )
  )
  (sdf_mlp): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=False)
    (1): Linear(in_features=128, out_features=64, bias=False)
    (2): Linear(in_features=64, out_features=1, bias=False)
  )
  (deform_mlp): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=False)
    (1): Linear(in_features=128, out_features=64, bias=False)
    (2): Linear(in_features=64, out_features=3, bias=False)
  )
  (feature_mlp): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=False)
    (1): Linear(in_features

In [ ]:
pred_sdfs, f_vs = get_pred_sdfs(sdf_model, tets_verts)
print(extract_tet(tets, d, 0.001)[0].shape)
print(sample_faces.shape)

torch.Size([24004])
torch.Size([20212, 3])


In [ ]:
gcn_pretrain(500, refine_model, pre_optimizer, d, tets_verts, tets, f_vs, sample_verts, sample_faces, 0.001)

100%|██████████| 500/500 [00:54<00:00,  9.20it/s]

Iteration 500 - loss: 0.011778036132454872


In [ ]:
refine_vars = [p for _, p in refine_model.named_parameters()]
refine_optimizer = torch.optim.Adam(pre_vars, lr=lr)
refine_scheduler = torch.optim.lr_scheduler.LambdaLR(pre_optimizer, lr_lambda=lambda x: max(0.0, 10**(-x*0.0002))) # LR decay over time

In [ ]:
gcn_train(gcn_iterations, refine_model, refine_optimizer, refine_scheduler, d, tets_verts, tets, f_vs, sample_verts, sample_faces, 0.001)

Iteration 0 - loss: 58331108.0, # of mesh vertices: 45972, # of mesh faces: 92280
Iteration 100 - loss: 5686.1953125, # of mesh vertices: 32120, # of mesh faces: 64288
Iteration 200 - loss: 2095.136962890625, # of mesh vertices: 37072, # of mesh faces: 74232
Iteration 300 - loss: 1226.7706298828125, # of mesh vertices: 40104, # of mesh faces: 80344
Iteration 400 - loss: 826.2510986328125, # of mesh vertices: 41646, # of mesh faces: 83456
Iteration 500 - loss: 606.0220336914062, # of mesh vertices: 42756, # of mesh faces: 85712
Iteration 600 - loss: 483.702392578125, # of mesh vertices: 43730, # of mesh faces: 87688
Iteration 700 - loss: 403.935546875, # of mesh vertices: 43978, # of mesh faces: 88204
Iteration 800 - loss: 345.98272705078125, # of mesh vertices: 44520, # of mesh faces: 89300
Iteration 900 - loss: 304.2041320800781, # of mesh vertices: 45170, # of mesh faces: 90652


In [ ]:
test_gcn = GCN_Res(CONFIG_GCNRES).to(device)

t_v = [p for _, p in test_gcn.named_parameters()]
t_op = torch.optim.Adam(t_v, lr=lr)
t_sch = torch.optim.lr_scheduler.LambdaLR(t_op, lr_lambda=lambda x: max(0.0, 10**(-x*0.0002))) # LR decay over time

In [ ]:
gcn_pretrain(700, test_gcn, t_op, d, tets_verts, tets, f_vs, sample_verts, sample_faces, 0)

100%|██████████| 700/700 [01:21<00:00,  8.58it/s]

Iteration 700 - loss: 0.006069199647754431


In [ ]:
gcn_train_wsubdiv(gcn_iterations, test_gcn, t_op, t_sch, d, tets_verts, tets, f_vs, sample_verts, sample_faces, 0)

In [ ]:
"""
to-dos: volume subdivision

identify T_surf's neighbors: i.e. share a same edge

subdivide T_surf to perform another surface refinement
unsubdivded tet, i.e. not surface's neighbor, is dropped to save memory and computation

"""

#neightbor if share an edge
#identify edge based on surf_faces and adj matrix

"""
convert face_lists = [v1, v2, v3, v4] --> 
[
  [v1, v2, v3, E, o1],
  [v2, v3, v4, E, o2],
  [v1, v2, v4, E, o3],
  [v1, v3, v4, E, o4]
]

by torch combinations, E = tet index, o_i = face_i index
"""
def convert_face_lists(tets):
  face_idx = torch.tensor([1,2,3,4])
  tet_face_list = []
  for idx, tet in enumerate(tets):
    tet_idx = torch.full(4, idx)
    tet_faces = torch.combinations(tet, r=3)
    tet_faces = torch.cat((tet_faces, tet_idx, face_idx), dim=1)
    tet_face_list.append(tet_faces)

  tet_face_list = torch.stack(tet_face_list, dim=0)

  def compare(face_1, face_2):
    o1 = face_1[0]-face_2[0]
    o2 = face_1[1]-face_2[1]
    o3 = face_1[2]-face_2[2]

    if o1 <= 0 or (o1 == 0 and o2 < 0) or (o1 == 0 and o2 == 0 and o3 < 0):
      return -1
    elif o1 == 0 and o2 == 0 and o3 == 0:
      return 0
    else: 
      return 1

  sorted(tet_face_list, cmp=compare)
  return tet_face_list

def get_neighbor(surf_tet_verts, surf_tets, tet_verts, tets):
  tet_face_list = convert_face_lists(tets)
  

surf_idx, surf = extract_tet(tets, pred_sdfs, 0.003)

print(surf_idx)

tensor([ 83308,  83316,  83318,  ..., 216949, 216951, 216959], device='cuda:0')


In [ ]:
def get_faces(input):
  c = torch.combinations(torch.arange(input.size(1)), r=3)
  x = input[:,None].expand(-1,len(c),-1).cpu()
  idx = c[None].expand(len(x), -1, -1)
  x = x.gather(dim=2, index=idx)

  return x.view(-1, *x.shape[2:])

In [ ]:
a = get_faces(tets)
b = get_faces(tets[surf_idx])

for f in b:
  mask = a == f


NameError: ignored

# Visualization

In [ ]:
#Use pyngrok to access localhost:80 on Colab

!pip install pyngrok --quiet 
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2Hzzzh94FgOXssVkSP5Yffz8uYg_By2RMDZLTPx1aXakhYfH"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [ ]:
#generating a public url mapped to localhost 80
public_url = ngrok.connect(port=80, proto="http", options={"bind_tls": True, "local": True})
print("Tracking URL:", public_url)

Tracking URL: NgrokTunnel: "http://40b6-34-124-135-125.ngrok.io" -> "http://localhost:80"


In [ ]:
#Start Kaolin Dash3D on localhost:80 
!kaolin-dash3d --logdir=/content/drive/MyDrive/CV_DMTet/Logs --port=80

Dash3D server starting. Go to: http://localhost:80
2022-12-08 19:14:38,232|    INFO|kaolin.visualize.timelapse| No checkpoints found for type pointcloud: no files matched pattern pointcloud*.usd in /content/drive/MyDrive/CV_DMTet/Logs
2022-12-08 19:14:38,236|    INFO|kaolin.visualize.timelapse| No checkpoints found for type voxelgrid: no files matched pattern voxelgrid*.usd in /content/drive/MyDrive/CV_DMTet/Logs
2022-12-08 19:14:41,667|    INFO|kaolin.visualize.timelapse| No checkpoints found for type pointcloud: no files matched pattern pointcloud*.usd in /content/drive/MyDrive/CV_DMTet/Logs
2022-12-08 19:14:41,673|    INFO|kaolin.visualize.timelapse| No checkpoints found for type voxelgrid: no files matched pattern voxelgrid*.usd in /content/drive/MyDrive/CV_DMTet/Logs
2022-12-08 19:14:41,694|    INFO| tornado.access| 200 GET / (127.0.0.1) 40.24ms
2022-12-08 19:14:42,142|    INFO| tornado.access| 200 GET /static/thirdparty.css (127.0.0.1) 3.64ms
2022-12-08 19:14:42,145|    INFO| tor

# Prev Code

In [ ]:
# from tqdm import tqdm

# # MLP + Positional Encoding
# class Decoder(torch.nn.Module):
#     def __init__(self, input_dims = 3, internal_dims = 128, output_dims = 4, hidden = 5, multires = 2):
#         super().__init__()
#         self.embed_fn = None
#         if multires > 0:
#             embed_fn, input_ch = get_embedder(multires)
#             self.embed_fn = embed_fn
#             input_dims = input_ch

#         # net = (torch.nn.Linear(input_dims, internal_dims, bias=False), torch.nn.ReLU())
#         # for i in range(hidden-1):
#         #     net = net + (torch.nn.Linear(internal_dims, internal_dims, bias=False), torch.nn.ReLU())
#         # net = net + (torch.nn.Linear(internal_dims, output_dims, bias=False),)
#         # self.net = torch.nn.Sequential(*net)

#         self.net = torch.nn.Sequential(
#             torch.nn.Linear(input_dims, 256, bias=False),
#             torch.nn.ReLU(),
#             torch.nn.Linear(256, 256, bias=False),
#             torch.nn.ReLU(),
#             torch.nn.Linear(256, 128, bias=False),
#             torch.nn.ReLU(),
#             torch.nn.Linear(128, 64, bias=False),
#             torch.nn.ReLU(),
#             torch.nn.Linear(64, output_dims, bias=False),
#         )

#     def forward(self, p):
#         if self.embed_fn is not None:
#             p = self.embed_fn(p)
#         print(p.shape)
#         out = self.net(p)
#         return out

#     def pre_train_sphere(self, iter):
#         print ("Initialize SDF to sphere")
#         loss_fn = torch.nn.MSELoss()
#         optimizer = torch.optim.Adam(list(self.parameters()), lr=1e-4)

#         for i in tqdm(range(iter)):
#             p = torch.rand((1024,3), device='cuda') - 0.5
#             ref_value  = torch.sqrt((p**2).sum(-1)) - 0.3
#             output = self(p)
#             loss = loss_fn(output[...,0], ref_value)
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#         print("Pre-trained MLP", loss.item())


# # Positional Encoding from https://github.com/yenchenlin/nerf-pytorch/blob/1f064835d2cca26e4df2d7d130daa39a8cee1795/run_nerf_helpers.py
# class Embedder:
#     def __init__(self, **kwargs):
#         self.kwargs = kwargs
#         self.create_embedding_fn()
        
#     def create_embedding_fn(self):
#         embed_fns = []
#         d = self.kwargs['input_dims']
#         out_dim = 0
#         if self.kwargs['include_input']:
#             embed_fns.append(lambda x : x)
#             out_dim += d
            
#         max_freq = self.kwargs['max_freq_log2']
#         N_freqs = self.kwargs['num_freqs']
        
#         if self.kwargs['log_sampling']:
#             freq_bands = 2.**torch.linspace(0., max_freq, steps=N_freqs)
#         else:
#             freq_bands = torch.linspace(2.**0., 2.**max_freq, steps=N_freqs)
            
#         for freq in freq_bands:
#             for p_fn in self.kwargs['periodic_fns']:
#                 embed_fns.append(lambda x, p_fn=p_fn, freq=freq : p_fn(x * freq))
#                 out_dim += d
                    
#         self.embed_fns = embed_fns
#         self.out_dim = out_dim
        
#     def embed(self, inputs):
#         return torch.cat([fn(inputs) for fn in self.embed_fns], -1)

# def get_embedder(multires):
#     embed_kwargs = {
#                 'include_input' : True,
#                 'input_dims' : 3,
#                 'max_freq_log2' : multires-1,
#                 'num_freqs' : multires,
#                 'log_sampling' : True,
#                 'periodic_fns' : [torch.sin, torch.cos],
#     }
    
#     embedder_obj = Embedder(**embed_kwargs)
#     embed = lambda x, eo=embedder_obj : eo.embed(x)
#     return embed, embedder_obj.out_dim

In [ ]:
# test_model = Decoder(multires=2).to(device)
# pred = test_model(tets_verts)

In [ ]:
# def sdf_train(iterations, model, optimizer):
#   model.eval()
#   for it in range(iterations):
#       pred = model(tet_verts) # predict SDF and per-vertex deformation
#       sdf, deform = pred[:,0], pred[:,1:]
#       verts_deformed = tet_verts + torch.tanh(deform) / grid_res # constraint deformation to avoid flipping tets
#       mesh_verts, mesh_faces = kaolin.ops.conversions.marching_tetrahedra(verts_deformed.unsqueeze(0), tets, sdf.unsqueeze(0)) # running MT (batched) to extract surface mesh
#       mesh_verts, mesh_faces = mesh_verts[0], mesh_faces[0]
#       loss = loss_f(mesh_verts, mesh_faces, sample_verts, sample_faces, it)
#       optimizer.zero_grad()
#       loss.backward()
#       optimizer.step()
#       scheduler.step()
#       if (it) % save_every == 0 or it == (iterations - 1): 
#           print ('Iteration {} - loss: {}, # of mesh vertices: {}, # of mesh faces: {}'.format(it, loss, mesh_verts.shape[0], mesh_faces.shape[0]))
#           sign = kaolin.ops.mesh.check_sign(sample_verts.unsqueeze(0).to(device), sample_faces.to(device), verts_deformed.unsqueeze(0))[0]
#           print(sign[sign == True])
#           # save reconstructed mesh
#           timelapse.add_mesh_batch(
#               iteration=it+1,
#               category='extracted_mesh',
#               vertices_list=[mesh_verts.cpu()],
#               faces_list=[mesh_faces.cpu()]
#           )